In [ ]:
# imports

import os
import re
import math
import random
from datetime import datetime
import numpy as np
from typing import Optional
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from huggingface_hub import login
from tqdm import tqdm
import matplotlib.pyplot as plt
from datasets import load_dataset, Dataset, DatasetDict
from concurrent.futures import ProcessPoolExecutor
from transformers import AutoTokenizer
from items_qwen import Item, read_dataset
import pickle

In [ ]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)
PHI3 = "microsoft/Phi-3-medium-4k-instruct"
GEMMA = "google/gemma-2-9b-it"
QWEN = "Qwen/Qwen2-7B"
LLAMA = "meta-llama/Meta-Llama-3.1-8B"

In [ ]:
%matplotlib inline

In [ ]:
# Constants

BASE_MODEL = GEMMA

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

for i in range(1,1000):
    text = str(i)
    tok = tokenizer.encode(text, add_special_tokens=False)
    print(f"{text}={tok}")


In [ ]:
dataset_names = [
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
]

In [ ]:
# start = datetime.now()
# items = read_dataset("Appliances")
# finish = datetime.now()
# print(f"Completed in {(finish-start).total_seconds()/60:.1f} mins")

In [ ]:
# print(items[10000].prompt)

In [ ]:
start = datetime.now()
items = []
with ProcessPoolExecutor(max_workers=6) as pool:
    for results in pool.map(read_dataset, dataset_names):
        items.extend(results)
finish = datetime.now()
print(f"Completed in {(finish-start).total_seconds()/60:.1f} mins")

In [ ]:
len(items)

In [ ]:
max(item.token_count for item in items)

In [ ]:
with open('items2.pkl', 'wb') as file:
    pickle.dump(items, file)

In [ ]:
with open('items2.pkl', 'rb') as file:
    items = pickle.load(file)

In [ ]:
len(items)

In [ ]:
# Let's investigate:

print(f"There are {len(items):,} items with prices\n")
print(items[2000000].prompt)

In [ ]:
# Plot the distribution of character count

lengths = [len(item.prompt) for item in items]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Length')
ax.set_ylabel('Count of items');
_ = ax.hist(lengths, rwidth=0.7, color="lightblue", bins=range(0, 1000, 50))

print(f"Average length is {sum(lengths)/len(lengths):,.1f} and highest length is {max(lengths):,}\n")

In [ ]:
# Plot the distribution of tokens

token_counts = [item.token_count for item in items]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="orange", bins=range(0, 200, 10))

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(items[1])

In [ ]:
# Plot the distribution of prices

prices = [float(item.price) for item in items]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Price ($)')
ax.set_ylabel('Count of items');
_ = ax.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 20))

print(f"Average price is ${sum(prices)/len(prices):.2f} and highest price is ${max(prices):,.2f}\n")

In [ ]:
from collections import Counter
category_counts = Counter()
for item in items:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# Create bar chart
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="purple")

# Customize the chart
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# Add value labels on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# Display the chart
plt.show()

In [ ]:
from collections import defaultdict
slots = defaultdict(list)
for item in items:
    slots[round(item.price)].append(item)

print(f"\nMinimum: {min([len(slot) for slot in slots.values()]):,}")

In [ ]:
np.random.seed(42)
random.seed(42)
sample = []
for i in range(1, 1000):
    slot = slots[i]
    if i>=240:
        sample.extend(slot)
    elif len(slot) <= 1200:
        sample.extend(slot)
    else:
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        weights = weights / np.sum(weights)
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)
len(sample)

In [ ]:
# Plot the distribution of prices

prices = [float(item.price) for item in sample]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# Create bar chart
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="lightgreen")

# Customize the chart
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# Add value labels on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# Display the chart
plt.show()

In [ ]:
report(sample[400000])

In [ ]:
category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

plt.figure(figsize=(12, 10))
plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)

# Add a circle at the center to create a donut chart (optional)
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Customize the chart
plt.title('Categories')

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')  

# Display the chart
plt.tight_layout()
plt.show()

In [ ]:
sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# Create the scatter plot
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# Add labels and title
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# Display the plot
plt.show()

In [ ]:
# Plot the distribution again to check it looks as expected

token_counts = [item.token_count for item in sample]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="purple", bins=range(0, 300, 10))

In [ ]:
report(sample[-2])

In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:400_000]
test = sample[400_000:402_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
# Plot the distribution of prices

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# del items, slots
# import gc
# gc.collect()

In [ ]:
import csv
with open('test.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:200]:
        writer.writerow([t.title, t.details, 0])

In [ ]:
human_predictions = []
with open('human.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[2]))

In [ ]:
average = sum(t.price for t in train)/len(train)
average

In [ ]:
class TestRunner:

    def __init__(self, predictor, data, title, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = RED if error>=40 else YELLOW if error>=20 else GREEN
        color_str = "red" if error>=40 else "yellow" if error>=20 else "green"
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color_str)
        print(f"{color}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = [e for e in self.errors if e<20]
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={len(hits)/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()
        return self

In [ ]:
train_average = sum(t.price for t in train)/len(train)

def flat_predictor(item):
    return train_average

In [ ]:
runner = TestRunner(flat_predictor, test, "Flat Predictor").run()

In [ ]:
human_predictions = []
with open('human.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[2]))

def human_predictor(item):
    index = test.index(item)
    if index==-1:
        raise ValueError("Index not found")
    return human_predictions[index]

In [ ]:
runner = TestRunner(human_predictor, test, "Human Predictor").run()

In [ ]:
stop = set(['the', 'and', 'for', 'is', 'to', 'this', 'with', 'a', 'of', 'your', 'are', 'in','from', 'you', 'or', 'an', 'on', 'by'])

def words(item):
  text = f"{item.title} {item.details}"
  text = re.sub(r'[\(\)\[\]\{\},\'"\- \s]+', ' ', text)
  words = text.strip().lower().split(' ')
  filtered = [word for word in words if word not in stop]
  return " ".join(filtered)

In [ ]:
documents = [words(item) for item in train]
from collections import Counter
count = Counter()
for doc in documents:
    ws = doc.split(" ")
    for w in ws:
        count[w]+=1
count.most_common(30)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np

np.random.seed(42)

labels = np.array([float(item.price) for item in train])

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

regressor = LinearRegression()
regressor.fit(X, labels)

In [ ]:
def linear_regression_predictor(item):
    np.random.seed(42)
    x = vectorizer.transform([words(item)])
    return max(regressor.predict(x)[0], 0)

In [ ]:
runner1 = TestRunner(linear_regression_predictor, test, "Linear Regression").run()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR

np.random.seed(42)

labels = np.array([float(item.price) for item in train])

vectorizer = TfidfVectorizer(max_features=20)
X = vectorizer.fit_transform(documents)

regressor = SVR(kernel='linear')
regressor.fit(X, labels)

In [ ]:
def svr_predictor(item):
    np.random.seed(42)
    x = vectorizer.transform([words(item)])
    return max(regressor.predict(x)[0], 0)

In [ ]:
runner2 = TestRunner(svr_predictor, test, "SVR Accuracy").run()

In [ ]:
def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

In [ ]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
def gpt_predictor(item):
    response = openai.chat.completions.create(
        model=OPENAI_MODEL, #"gpt-4o-2024-08-06", 
        messages=messages_for(item),
        seed=42,
        max_tokens=6
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
runner = TestRunner(gpt_predictor, test, "GPT-4o-mini Prediction Accuracy").run()

In [ ]:
def frontier_predictor(item):
    response = openai.chat.completions.create(
        model="gpt-4o-2024-08-06", 
        messages=messages_for(item),
        seed=42,
        max_tokens=6
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
runner = TestRunner(frontier_predictor, test, "GPT-4o Frontier Prediction").run()

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
test_prompts[2]

In [ ]:
# Create a Dataset from the lists
train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
DATASET_NAME = "ed-donner/pricer-data"
dataset.push_to_hub(DATASET_NAME, private=True)